# 2. Data preparation for Clustering
For our clustering analysis we need to prepare the data accordingly. Following on from our data analysis we want to try to cluster on the profit margin of participants against the number of times the subjects picked their most common deck choice or their average choice. We will then combine this with a scatter plot showing the study each subject was a part of and see what information we can gather from this. We will be looking at age demographies more so but also look to combine this with the amount of cards that pay out in each study and gender breakdowns also. To do this we need to create appropriate CSV files that we can then use for clustering. 

In [73]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [74]:
index95 = pd.read_csv('data/index_95.csv')
index100 = pd.read_csv('data/index_100.csv')
index150 = pd.read_csv('data/index_150.csv')
win95 = pd.read_csv('data/wi_95.csv')
win100 = pd.read_csv('data/wi_100.csv')
win150 = pd.read_csv('data/wi_150.csv')
loss95 = pd.read_csv('data/lo_95.csv')
loss100 = pd.read_csv('data/lo_100.csv')
loss150 = pd.read_csv('data/lo_150.csv')
choice95 = pd.read_csv('data/choice_95.csv')
choice100 = pd.read_csv('data/choice_100.csv')
choice150 = pd.read_csv('data/choice_150.csv')

### Creating margin csv files

In [75]:
columnnames95 = [f'Trial{num}' for num in range(1,96)]
wins95 = win95
wins95 = wins95.set_axis(columnnames95, axis=1)
wins95.head()

,Trial1,Trial2,Trial3,Trial4,Trial5,Trial6,Trial7,Trial8,Trial9,Trial10,...,Trial86,Trial87,Trial88,Trial89,Trial90,Trial91,Trial92,Trial93,Trial94,Trial95
Subj_1,100,100,100,100,100,100,100,100,100,100,...,50,50,50,50,50,50,50,50,50,50
Subj_2,100,100,50,100,100,100,100,100,100,100,...,50,100,100,100,100,100,50,50,50,50
Subj_3,50,50,50,100,100,100,100,100,100,100,...,100,100,100,50,50,50,50,50,50,50
Subj_4,50,50,100,100,100,100,100,50,100,100,...,100,50,50,50,50,50,50,50,50,50
Subj_5,100,100,50,50,50,100,100,100,100,100,...,50,50,50,50,50,50,50,50,50,50


In [76]:
losses95 = loss95
losses95 = losses95.set_axis(columnnames95, axis=1)
losses95.head()

,Trial1,Trial2,Trial3,Trial4,Trial5,Trial6,Trial7,Trial8,Trial9,Trial10,...,Trial86,Trial87,Trial88,Trial89,Trial90,Trial91,Trial92,Trial93,Trial94,Trial95
Subj_1,0,0,0,0,0,0,0,0,-1250,0,...,0,0,0,0,0,0,0,-250,0,0
Subj_2,0,0,0,0,0,0,0,0,0,0,...,-50,-300,0,-350,0,0,0,0,0,-25
Subj_3,0,0,0,0,0,0,0,-150,0,0,...,0,0,0,0,0,0,-250,0,0,0
Subj_4,0,0,0,0,-150,0,0,0,0,0,...,0,-50,0,-50,-50,0,-25,0,0,0
Subj_5,0,0,0,0,0,0,-150,0,0,0,...,-75,0,0,0,0,0,0,0,0,0


In [77]:
df95_sum = wins95.add(losses95, fill_value=0)
df95_sum.head()

,Trial1,Trial2,Trial3,Trial4,Trial5,Trial6,Trial7,Trial8,Trial9,Trial10,...,Trial86,Trial87,Trial88,Trial89,Trial90,Trial91,Trial92,Trial93,Trial94,Trial95
Subj_1,100,100,100,100,100,100,100,100,-1150,100,...,50,50,50,50,50,50,50,-200,50,50
Subj_2,100,100,50,100,100,100,100,100,100,100,...,0,-200,100,-250,100,100,50,50,50,25
Subj_3,50,50,50,100,100,100,100,-50,100,100,...,100,100,100,50,50,50,-200,50,50,50
Subj_4,50,50,100,100,-50,100,100,50,100,100,...,100,0,50,0,0,50,25,50,50,50
Subj_5,100,100,50,50,50,100,-50,100,100,100,...,-25,50,50,50,50,50,50,50,50,50


In [78]:
profit95 = df95_sum.sum(axis=1)
profit95df = pd.DataFrame(data=profit95)
profit95df.rename(columns={0: 'Margin'}, inplace=True)
profit95df.head()

,Margin
Subj_1,1150
Subj_2,-675
Subj_3,-750
Subj_4,-525
Subj_5,100


In [79]:
choice95.head()

,Choice_1,Choice_2,Choice_3,Choice_4,Choice_5,Choice_6,Choice_7,Choice_8,Choice_9,Choice_10,...,Choice_86,Choice_87,Choice_88,Choice_89,Choice_90,Choice_91,Choice_92,Choice_93,Choice_94,Choice_95
Subj_1,2,2,2,2,2,2,2,2,2,1,...,4,4,4,4,4,4,4,4,4,4
Subj_2,1,2,3,2,2,2,2,2,2,2,...,3,1,1,1,2,2,3,4,4,3
Subj_3,3,4,3,2,2,1,1,1,1,2,...,2,2,2,4,4,4,4,4,4,4
Subj_4,4,3,1,1,1,2,2,3,2,2,...,2,3,3,3,3,3,3,4,4,4
Subj_5,1,2,3,4,3,1,1,2,2,2,...,3,3,4,4,3,4,4,4,4,4


In [80]:
most_common_count = choice95.apply(pd.Series.value_counts, axis=1)
most_common_count = most_common_count.max(axis=1)
profit95df['Most Common Choice Picked'] = most_common_count
profit95df.head()

,Margin,Most Common Choice Picked
Subj_1,1150,71
Subj_2,-675,33
Subj_3,-750,38
Subj_4,-525,38
Subj_5,100,46


In [81]:
mode95 = choice95.mode(axis=1)
mode95.rename(columns={0: 'Most Common Choice'}, inplace=True)

In [82]:
profit95df['Most Common Choice'] = mode95['Most Common Choice'].values

In [83]:
profit95df['Study'] = index95['Study'].values
profit95df.head()

,Margin,Most Common Choice Picked,Most Common Choice,Study
Subj_1,1150,71,4,Fridberg
Subj_2,-675,33,4,Fridberg
Subj_3,-750,38,4,Fridberg
Subj_4,-525,38,4,Fridberg
Subj_5,100,46,4,Fridberg


In [84]:
mean95 = choice95.mean(axis=1)
mean95df = pd.DataFrame(data=mean95)
mean95df.rename(columns={0: 'Average Choice'}, inplace=True)
profit95df['Average Choice'] = mean95df['Average Choice'].values
profit95df.head()

,Margin,Most Common Choice Picked,Most Common Choice,Study,Average Choice
Subj_1,1150,71,4,Fridberg,3.400000
Subj_2,-675,33,4,Fridberg,2.568421
Subj_3,-750,38,4,Fridberg,2.778947
Subj_4,-525,38,4,Fridberg,2.810526
Subj_5,100,46,4,Fridberg,3.021053


In [85]:
profit95df.to_csv('Data/cleaned95.csv')

## We now do this for the 100 trial and 150 trial experiments

In [86]:
columnnames100 = [f'Trial{num}' for num in range(1,101)]
wins100 = win100
wins100 = wins100.set_axis(columnnames100, axis=1)

In [87]:
losses100 = loss100
losses100 = losses100.set_axis(columnnames100, axis=1)

In [88]:
df100_sum = wins100.add(losses100, fill_value=0)

In [89]:
profit100 = df100_sum.sum(axis=1)
profit100df = pd.DataFrame(data=profit100)
profit100df.rename(columns={0: 'Margin'}, inplace=True)

In [90]:
profit100df['Study'] = index100['Study'].values

In [91]:
mode100 = choice100.mode(axis=1)
mode100.rename(columns={0: 'Most Common Choice'}, inplace=True)
profit100df['Most Common Choice'] = mode100['Most Common Choice'].values

In [92]:
profit100df['Most Common Choice'].value_counts()

2.0    221
4.0    171
3.0     98
1.0     14
Name: Most Common Choice, dtype: int64

In [93]:
profit100df['Most Common Choice'] = profit100df['Most Common Choice'].astype('int64')

In [94]:
most_common_count100 = choice100.apply(pd.Series.value_counts, axis=1)
most_common_count100 = most_common_count100.max(axis=1)
profit100df['Most Common Choice Picked'] = most_common_count100
profit100df.head()

,Margin,Study,Most Common Choice,Most Common Choice Picked
Subj_1,-1800,Horstmann,2,42
Subj_2,-800,Horstmann,2,35
Subj_3,-450,Horstmann,2,42
Subj_4,1200,Horstmann,4,35
Subj_5,-1300,Horstmann,2,31


In [95]:
mean100 = choice100.mean(axis=1)
mean100df = pd.DataFrame(data=mean100)
mean100df.rename(columns={0: 'Average Choice'}, inplace=True)
profit100df['Average Choice'] = mean100df['Average Choice'].values

In [96]:
profit100df.to_csv('Data/cleaned100.csv')

Lastly, we take the 150 trial data.

In [97]:
columnnames150 = [f'Trial{num}' for num in range(1,151)]
wins150 = win150
wins150 = wins150.set_axis(columnnames150, axis=1)

In [98]:
losses150 = loss150
losses150 = losses150.set_axis(columnnames150, axis=1)

In [99]:
df150_sum = wins150.add(losses150, fill_value=0)

In [100]:
profit150 = df150_sum.sum(axis=1)
profit150df = pd.DataFrame(data=profit150)
profit150df.rename(columns={0: 'Margin'}, inplace=True)

In [101]:
profit150df['Study'] = index150['Study'].values

In [102]:
mode150 = choice150.mode(axis=1)
mode150.rename(columns={0: 'Most Common Choice'}, inplace=True)
profit150df['Most Common Choice'] = mode150['Most Common Choice'].values

In [103]:
most_common_count150 = choice150.apply(pd.Series.value_counts, axis=1)
most_common_count150 = most_common_count150.max(axis=1)
most_common_count150 = most_common_count150.astype('int64')
profit150df['Most Common Choice Picked'] = most_common_count150
profit150df.head()

,Margin,Study,Most Common Choice,Most Common Choice Picked
Subj_1,-550,Steingroever2011,1.0,46
Subj_2,-1600,Steingroever2011,2.0,57
Subj_3,900,Steingroever2011,4.0,88
Subj_4,2200,Steingroever2011,4.0,111
Subj_5,1900,Steingroever2011,4.0,135


In [104]:
mean150 = choice150.mean(axis=1)
mean150df = pd.DataFrame(data=mean150)
mean150df.rename(columns={0: 'Average Choice'}, inplace=True)
profit150df['Average Choice'] = mean150df['Average Choice'].values

In [105]:
profit150df.to_csv('Data/cleaned150.csv')

In [106]:
merged95_150 = pd.concat([profit95df, profit150df])

In [114]:
mergedall = pd.concat([merged95_150, profit100df])
mergedall['Most Common Choice'] = mergedall['Most Common Choice'].astype('int64')
mergedall

,Margin,Most Common Choice Picked,Most Common Choice,Study,Average Choice
Subj_1,1150,71,4,Fridberg,3.400000
Subj_2,-675,33,4,Fridberg,2.568421
Subj_3,-750,38,4,Fridberg,2.778947
Subj_4,-525,38,4,Fridberg,2.810526
Subj_5,100,46,4,Fridberg,3.021053
...,...,...,...,...,...
Subj_500,75,29,2,Worthy,2.630000
Subj_501,600,44,3,Worthy,2.840000
Subj_502,-1525,32,2,Worthy,2.380000
Subj_503,-750,27,1,Worthy,2.460000


For some of our comparisons we may want to draw on in our k-means clusters we need to change our study values from strings to integers. After plotting our k-means algorithm this will allow us to plot a scatter plot comprising of the different studies which will be colour coded based on their numbers here.

In [117]:
replacements_study = {
  r'Fridberg': 0,  
  r'Horstmann': 1,
  r'Kjome': 2,
  r'Maia': 3,
  r'SteingroverInPrep': 4,
  r'Premkumar': 5,
  r'Wood': 6,
  r'Worthy': 7,
  r'Steingroever2011': 8,
  r'Wetzels': 9,  
}

mergedall['StudyNumber'] = mergedall.Study.replace(replacements_study, regex=True)
mergedall = mergedall.drop(columns=['Study'])
mergedall

,Margin,Most Common Choice Picked,Most Common Choice,Average Choice,StudyNumber
Subj_1,1150,71,4,3.400000,0
Subj_2,-675,33,4,2.568421,0
Subj_3,-750,38,4,2.778947,0
Subj_4,-525,38,4,2.810526,0
Subj_5,100,46,4,3.021053,0
...,...,...,...,...,...
Subj_500,75,29,2,2.630000,7
Subj_501,600,44,3,2.840000,7
Subj_502,-1525,32,2,2.380000,7
Subj_503,-750,27,1,2.460000,7


In [118]:
mergedall.to_csv('Data/cleaned_all.csv')

## Standardize our Data
To work best with our k-means algorithm we choose to standardize our values in our joined dataset. This is because the k-means algorithm is a distance based algorithm, calculating the similarity between points based on distance. This gives the data a mean of 0 and standard deviation of 1 and gives common ground between features which would use different values such as our margin and average choice columns.

In [129]:
scaler = preprocessing.StandardScaler().fit(mergedall)
X_scaled = scaler.transform(mergedall)
X_scaled.std(axis=0)

array([1., 1., 1., 1., 1.])

In [130]:
standard_all = pd.DataFrame(X_scaled)

In [131]:
standard_all = standard_all.rename(columns={0:'Margin', 1: 'Most Common Choice Picked', 2: 'Most Common Choice', 3: 'Average Choice',
                                           4: 'StudyNumber'})

In [132]:
standard_all

,Margin,Most Common Choice Picked,Most Common Choice,Average Choice,StudyNumber
0,1.044988,1.062672,1.234405,2.271920,-1.609380
1,-0.414346,-0.804770,1.234405,-0.410317,-1.609380
2,-0.474318,-0.559054,1.234405,0.268730,-1.609380
3,-0.294400,-0.559054,1.234405,0.370587,-1.609380
4,0.205371,-0.165908,1.234405,1.049635,-1.609380
...,...,...,...,...,...
612,0.185381,-1.001343,-0.923181,-0.211696,0.952696
613,0.605189,-0.264195,0.155612,0.465654,0.952696
614,-1.094035,-0.853913,-0.923181,-1.018065,0.952696
615,-0.474318,-1.099629,-2.001975,-0.760027,0.952696


In [134]:
standard_all.to_csv('data/standardized_all.csv')